In [0]:
from bs4 import BeautifulSoup
from lxml import etree
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re
import csv

In [0]:
#getting all the pages with listings
pages_url = list(map(lambda pag: 'https://www.glassdoor.com/Job/portugal-data-jobs-SRCH_IL.0,8_IN195_KE9,13_IP' + pag + '.htm?radius=25', [str(pages) for pages in list(range(1,31))]))

#getting all the jobs links from all the pages
def grab_links(pages_url):
  job_links = []
  for url in pages_url:
    response_url = requests.get(url, headers={'user-agent': 'Mozilla/5.0'})
    soup_url = BeautifulSoup(response_url.content, 'lxml')
    links_all = [element.get('href') for element in soup_url.find_all('a')]
    links = set(list(filter(lambda x: x.startswith('/partner') if x else False, links_all)))
    job_links += links
  return job_links

description_url = grab_links(pages_url)

#joining the link to whole url

description_links = set(list(map(lambda x: 'https://www.glassdoor.com' + x, description_url)))

number_of_links = len(description_links) # 2017

In [0]:

# access each job description url to get list of soups
def get_info(links):
    positions = []
    companies = []
    cities = []
    skills = []
    count = 0   
    for link in links:
        response_link = requests.get(link, headers={'user-agent': 'Mozilla/5.0'})
        soup_link = BeautifulSoup(response_link.content, 'lxml')
        position = [element.text.strip() for element in soup_link.find_all('h2', attrs = {'class':'noMargTop margBotXs strong'})]
        company = [element.text.strip() for element in soup_link.find_all('span', attrs = {'class':'strong ib'})]
        city = [element.text.replace('\xa0', '').replace('–', '').strip() for element in soup_link.find_all('span', attrs = {'class':'subtle ib'})]
        skill = [element.text for element in soup_link.find_all('li')][29:-28]
        positions += position
        companies += company
        cities += city
        skills += skill
        count += 1
        print('Processing... {} of 2023'.format(count) )
    return [positions, companies, cities, skills]

#descriptions_response = list(map(lambda x: requests.get(x, headers={'user-agent': 'Mozilla/5.0'}), description_links)
info_list = get_info(description_links)

Processing... 1 of 2023
Processing... 2 of 2023
Processing... 3 of 2023
Processing... 4 of 2023
Processing... 5 of 2023
Processing... 6 of 2023
Processing... 7 of 2023
Processing... 8 of 2023
Processing... 9 of 2023
Processing... 10 of 2023
Processing... 11 of 2023
Processing... 12 of 2023
Processing... 13 of 2023
Processing... 14 of 2023
Processing... 15 of 2023
Processing... 16 of 2023
Processing... 17 of 2023
Processing... 18 of 2023
Processing... 19 of 2023
Processing... 20 of 2023
Processing... 21 of 2023
Processing... 22 of 2023
Processing... 23 of 2023
Processing... 24 of 2023
Processing... 25 of 2023
Processing... 26 of 2023
Processing... 27 of 2023
Processing... 28 of 2023
Processing... 29 of 2023
Processing... 30 of 2023
Processing... 31 of 2023
Processing... 32 of 2023
Processing... 33 of 2023
Processing... 34 of 2023
Processing... 35 of 2023
Processing... 36 of 2023
Processing... 37 of 2023
Processing... 38 of 2023
Processing... 39 of 2023
Processing... 40 of 2023
Processin

In [0]:
info_list

In [0]:
job = info_list[0]
comp = set(info_list[1])
city = info_list[2]
skil = set(info_list[3])

#cleaning skill text
words_skill = ' '.join(skil).replace(',','').replace('.','').replace('(', '').replace(')', '').replace('/','').replace('/', '').replace('-', '').lower()


In [0]:
# years of experience required

experience_years = re.findall('\d.y', words_skill)
experience_plus = re.findall('\d\+', words_skill)
exp_y_int = [int(n[0]) for n in experience_years]
exp_p_int = [int(n[0]) for n in experience_plus]
experience = exp_y_int + exp_p_int
average = round(sum(experience)/len(experience), 2) # 3.46
informed_exp = len(experience) # 208


In [0]:
# bag of words

stopwords =  ['a', 'as', 'about', 'after', 'all', 'also', 'always', 'am', 'an', 'and', 'any', 'are', 'at', 'be', 'been', 'being', 'but', 'by', 'came', 'can', 'cant', 'come', 'could', 'did', "didn't", 'do', 'does', "doesn't",'doing', "don't", 'else', 'for', 'from', 'get', 'give', 'goes', 'going', 'had', 'happen', 'has', 'have', 'having', 'how', 'i', 'if', 'ill', "i'll", "i'm",'in', 'into', 'is', "isn't", 'it', 'its', "it's", "I've", 'just', 'keep', 'let', 'like', 'made', 'make', 'many', 'may', 'me', "mean", "more", "mean", "more", "most", "much", "no", "not", "now", 'on', "of", "only", "or", "our", "really", "say", "see", "some", "something", "most", "much", "no", "not", "now", "of", "only", "or", "our", "really", "say", "see", "some", "something", "take", "tell", "than", "that", "the", "their", "them", "then", "they", "thing", "this", "to", "try", "up", "us", "use", "used", "uses", "very", "want", "was", "way", "we", "what", "when", "where", "which", "who", "why", "will", "with", "without", "wont", "you", "your", "you're"]

skill_lst = words_skill.split()

for s in stopwords:
    for w in skill_lst:
        if w == s:
            skill_lst.remove(w)

bag_of_words = {k:skill_lst.count(k) for k in set(skill_lst) if len(k) > 1}
bag_of_words = {k:v for k,v in bag_of_words.items() if v > 20}
df_words = pd.DataFrame(bag_of_words, index=[0]).T.rename(columns={0:'#'})
df_words

In [0]:
df_words.sort_values('#', ascending=False).head(50)

In [0]:
# remote jobs
len(re.findall('remote', words_skill))

In [0]:
sorted_bag = sorted(bag_of_words.items(), key=lambda kv: kv[1], reverse=True)

In [0]:
# count cities
bag_of_cities = {k:' '.join(city).count(k) for k in set(city)}

bag_of_cities

In [0]:
#bag of jobs

job_picks = ['senior', 'python', 'intermediate', 'analyst', 'analysis', 'analytics', 'analista', 'data', 'architect', 'database', 'hackers', 'tech', 'machine', 'learning', 'it', 'ai', 'bi', 'sql', 'software', 'programmer', 'programador', 'scientist', 'sciente', 'engineer', 'engineering', 'developer', 'development', 'artificial', 'intelligence', 'junior']

#job = ['data analyst', 'nothing', 'nothing ai']

jobs = [word.lower() for j in job for word in j.split()]
jobs_selected = [word for word in jobs if word in job_picks]
jobs_selected

In [0]:
bag_of_jobs = {word:jobs_selected.count(word) for word in set(jobs_selected)}
  
df_jobs = pd.DataFrame(bag_of_jobs, index=[0])
df_jobs_sim = df_jobs[['ai', 'python',	'junior',	'scientist',	'machine',	'learning', 'hackers',	'software',	'database',	'bi', 'it', 'tech',	'senior',	'sql',	'architect',	'intelligence', 'data']].copy()

#df_analyst = [[col] for col in df_jobs if col.startswith('anal')]
df_jobs_sim['analyst'] = df_jobs.apply(lambda x: x['analysis'] + x['analista'] + x['analytics'] + x['analyst'], axis=1)
df_jobs_sim['engineer'] = df_jobs.apply(lambda x: x['engineer'] + x['engineering'], axis=1)
df_jobs_sim['programmer'] = df_jobs.apply(lambda x: x['programmer'] + x['programador'], axis=1)
df_jobs_sim['developer'] = df_jobs.apply(lambda x: x['developer'] + x['development'], axis=1)
df = df_jobs_sim.T.rename(columns={0:'#'}).sort_values('#', ascending=False)
df

In [0]:
x, y = np.ogrid[:300, :300]

mask = (x - 150) ** 2 + (y - 150) ** 2 > 130 ** 2
mask = 255 * mask.astype(int)
text_jobs = ' '. join(jobs_selected)
wc_jobs = WordCloud(background_color="rgba(255, 255, 255, 0)", mode="RGBA", repeat=True, mask=mask)
wc_jobs.generate(text_jobs)

plt.axis("off")
plt.imshow(wc_jobs, interpolation="bilinear")
plt.show()

In [0]:
#SALARY COMPARISON
#getting all the pages with listings

url_dev = 'https://www.payscale.com/research/US/Job=Software_Developer/Salary'
url_ana = 'https://www.payscale.com/research/US/Job=Data_Analyst/Salary' 
url_sci = 'https://www.payscale.com/research/US/Job=Data_Scientist/Salary' 
url_bus = 'https://www.payscale.com/research/US/Job=Business_Analyst%2C_IT/Salary' 

urls = [url_dev, url_ana, url_sci, url_bus]
salary = []

for url in urls:
  response_url = requests.get(url)
  soup_url = BeautifulSoup(response_url.content, 'lxml')
  sal = [element.text[1:3] for element in soup_url.find_all('div', attrs = {'class':'percentile-chart__median'})]
  salary += sal

to_euros = list(map(lambda x: float(x) * 0.91, salary))

job_titles = ['Software Developer', 'Data Analyst', 'Data Scientist', 'Business Analyst']
salary_world = list(zip(job_titles, to_euros))
in_lisb = [21.975, 15.024, 29.226, 27.289]
salary_lisb = list(zip(job_titles, in_lisb))

In [0]:
df = pd.DataFrame({'USA': to_euros, 'Lisbon': in_lisb}, index=job_titles)
ax = df.plot.bar(rot=45)